# Fluid mixture calculations with SAFT-$\gamma$-Mie

In this notebook, the available calculations with ```saftgammamie``` EoS for fluid mixtures are illustrated.

First, all the needed modules are imported.

- numpy: numerical interface and work with arrays
- sgtpy: package with SAFT-$\gamma$-Mie EoS.

In [1]:
import numpy as np
from sgtpy import component, mixture, saftgammamie

Now, pure components are configured and created with the ```component``` function. Then, a mixture is created with them using the ```mixture``` function or by adding (`+`) pure components. The interaction parameters are set up with the ```mixture.saftgammamie``` method. Finally, the ```eos``` object is created with the ```saftgammamie``` function.

The ```eos``` object includes all the necessary methods to compute phase equilibria and interfacial properties using SAFT-$\gamma$-Mie EoS.

For this notebook, all calculations are exemplified for a mixture of ethanol and water.

In [2]:
# setting up pure components
ethanol = component(GC={'CH3':1, 'CH2OH':1})
water = component(GC={'H2O':1})

# creating mixture with mixture class function
mix = mixture(ethanol, water)

# creating mixture by adding pure components
mix = ethanol + water

# reading database
mix.saftgammamie()
eos = saftgammamie(mix)

You can compute bulk phase density (mol/m$^3$) at given composition (x), temperature (K), and pressure (Pa) using the ```eos.density``` method. If no initial guess is provided, the root volume is computed using Topliss's method.

You can set ```'L'``` for the liquid phase and ```'V'``` for the vapor phase.

In [3]:
# density computation with Topliss's method
x = np.array([0.2, 0.8])
T = 350.  # K
P = 1e5  # Pa

# density in mol/m3
rhol = eos.density(x, T, P, 'L')
rhol

38009.95519266561

If a good initial guess is available for the density root, you can supply it, and then Newton's method is used to compute the root. This is **faster** than using Topliss's method.

In [4]:
# density computation with Newton's method
x = np.array([0.2, 0.8])
T = 350.  # K
P = 1e5  # Pa
# density in mol/m3
eos.density(x, T, P, 'L', rho0=0.95*rhol)

38009.95519266112

You can verify that the root was computed correctly with the ```eos.pressure``` method. This method computes the pressure (Pa) at given composition (x), density (mol/m$^3$) and temperature (K).

In [5]:
# computed pressure in Pa
eos.pressure(x, rhol, T)

99999.99998777454

If the mixtures associates, you can compute the non-bonded sites fractions using the `eos.association_solver` method.

In [6]:
# Computing non-bonded association sites fraction
Xass = eos.association_solver(x, rhol, T)
Xass 

array([0.36650211, 0.14940489, 0.08740906, 0.2208706 ])

Additionally, you can verify the association site mass balance using the `eos.association_check` method. If the non-bonded association sites are correctly computed, this function should be zero.

In [7]:
# checking the objective function for computed non-boned association sites fraction
eos.association_check(x, rhol, T, Xass)

array([0.00000000e+00, 2.77555756e-17, 0.00000000e+00, 5.55111512e-17])

You can compute the chemical potential using the ```eos.muad``` method. This method requires the density vector and temperature as input.

In [8]:
# computing chemical potential (dimensionless = mu/RT)
rhoi = x * rhol 
eos.muad(rhoi, T)

array([-0.68847568, -0.79364019])

Similarly, you can compute the Helmholtz free energy density with the ```eos.a0ad```. This method also uses the density vector and temperature as inputs.

In [9]:
# computing chemical potential (results is a0/RT = mol/m^3)
rhoi = x * rhol
eos.a0ad(rhoi, T)

-29401.131901709283

For phase equilibria and stability calculations, you can compute the effective fugacity coefficient natural logarithm using the ```eos.logfugef``` method.

This method requires the composition (x), temperature (K), pressure (Pa), and the aggregation state of the fluid (```'L'``` or ```'V'```). Optionally, if available, you can supply an initial guess to solve the volume root.

This function returns the effective fugacity coefficient natural logarithm and the computed volume root.

In [10]:
# computing the effective fugacity coefficients and volume root (m^3/mol)
eos.logfugef(x, T, P, 'L', v0 = 1/rhol)

(array([ 0.65182068, -0.83963818]), 2.6308897101592854e-05)

When needed you can compute the fugacity coefficient of the mixture at the given composition (x), temperature (K), pressure (Pa), and aggregation state using the ```eos.logfugmix``` method.

If available, you also can supply an initial guess to solve the volume root.

This function returns the fugacity coefficient natural logarithm of the mixture and the found volume root.

In [11]:
eos.logfugmix(x, T, P, 'L', v0 = 1/rhol)

(-0.5413464106357493, 2.6308897101592854e-05)

You also can compute thermal derived properties, such as, residual Entropy (```eos.EntropyR```), residual Enthalpy (```eos.EnthalyR```), residual isochoric and isobaric heat capacities (```eos.CvR``` and ```eos.CpR```).

Furthermore, you can compute the speed of sound using the ```eos.speed_sound``` method. You will be required to supply the ideal gas contribution for the heat capacities at the given temperature (These can be obtained from DIPPR 801 database).

In [12]:
# residual entropy in J / mol K
eos.EntropyR(x, T, P, 'L', v0=1/rhol)

-112.52976432125453

In [13]:
# resigual enthalpy in J / mol
eos.EnthalpyR(x, T, P, 'L', v0=1/rhol)

-40960.76918175604

In [14]:
# residual isochoric heat capacity in J / mol K
eos.CvR(x, rhol, T)

38.19197389068879

In [15]:
# residual isobaric heat capacity in J / mol K
eos.CpR(x, T, P, 'L', v0=1/rhol)

47.8116913756324

In [16]:
# speed of sound in m/s (value can be improved if correlated ideal gas Cv and Cp are used)
eos.speed_sound(x, T, P, 'L', v0=1/rhol)

1842.6858610761158

For further information about a method just run: ```eos.method?```